In [66]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [286]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class Triangle(Point):
    def __init__(self, x, y, field_of_reproduce, reproduce_timer, max_age):
        super().__init__(x, y)
        self.field_of_reproduce = field_of_reproduce
        self.reproduce_timer = reproduce_timer
        self.timer = reproduce_timer
        self.max_age = max_age

    def reproduce(self, plants, entities):
        if self.reproduce_timer == 0:
            occ_points = []
            for entity in entities:
                dx = self.x - entity.x
                dy = self.y - entity.y
                if abs(dx) < self.field_of_reproduce and abs(dy) < self.field_of_reproduce:
                    occ_points.append((dx, dy))

            child_x = random.randint(self.x - self.field_of_reproduce, self.x + self.field_of_reproduce)
            child_y = random.randint(self.y - self.field_of_reproduce, self.y + self.field_of_reproduce)
            while (child_x, child_y) in occ_points or (child_x, child_y) == (self.x, self.y):
                child_x = random.randint(self.x - self.field_of_reproduce, self.x + self.field_of_reproduce)
                child_y = random.randint(self.y - self.field_of_reproduce, self.y + self.field_of_reproduce)

                self.reproduce_timer = self.timer

                entities.append(Triangle(child_x, child_y, self.field_of_reproduce, self.reproduce_timer, self.max_age))
                plants.append(Triangle(child_x, child_y, self.field_of_reproduce, self.reproduce_timer, self.max_age))
        else:
            self.reproduce_timer -= 1

    def is_alive(self):
        return self.max_age > 0


class Star(Point):
    def __init__(self, x, y, filed_of_view, run_length):
        super().__init__(x, y)
        self.hunger = random.randint(10, 30)
        self.is_hungry = False#random.choice([True, False])
        self.field_of_view = filed_of_view
        self.run_length = run_length

    def detect_prey(self, prey):
        detected = []
        for entity in prey:
            dx = self.x - entity.x
            dy = self.y - entity.y
            if abs(dx) < self.field_of_view and abs(dy) < self.field_of_view:
                detected.append(entity)
        return detected

    def detect_partner(self, predators):
        detected = []
        for entity in predators:
            dx = self.x - entity.x
            dy = self.y - entity.y
            if abs(dx) < self.field_of_view and abs(dy) < self.field_of_view and (self.x != entity.x and self.y != entity.y):
                detected.append(entity)
        return detected

    def move(self, plant, prey, predators, entities):
        move_x = 0
        move_y = 0
        min_x = 10 ** 6
        min_y = 10 ** 6
        if self.is_hungry:
            detected_prey = self.detect_prey(prey)
            pot_prey = None
            for entity in detected_prey:
                dx = self.x - entity.x
                dy = self.y - entity.y
                if abs(dx) < abs(min_x) and abs(dx) < abs(min_y):
                    min_x = dx
                    min_y = dy
                    pot_prey = entity
    
            if pot_prey is None:
                move_x, move_y = random.choice([(0, self.run_length), (self.run_length, 0), (0, -self.run_length), (-self.run_length, 0)])
                
            elif pot_prey and pot_prey.is_alive():
                move_x = - np.sign(min_x) * self.run_length
                move_y = - np.sign(min_y) * self.run_length

                if abs(dx) <= 1 and abs(dy) <= 1:
                    pot_prey.hunger = 0
                    self.is_hungry = False
                    self.hunger = 5
                    self.x = pot_prey.x
                    self.y = pot_prey.y
        else:
            detected_partners = self.detect_partner(predators)
            pot_partner = None
            for entity in detected_partners:
                dx = self.x - entity.x
                dy = self.y - entity.y
                if abs(dx) < abs(min_x) and abs(dx) < abs(min_y):
                    min_x = dx
                    min_y = dy
                    pot_partner = entity

            if pot_partner is None:
                move_x, move_y = random.choice([(0, self.run_length), (self.run_length, 0), (0, -self.run_length), (-self.run_length, 0)])
                
            elif pot_partner and pot_partner.is_alive():
                move_x = - np.sign(min_x) * self.run_length
                move_y = - np.sign(min_y) * self.run_length

                if abs(dx) <= 1 and abs(dy) <= 1:
                    self.reproduce(predators, entities)
        start_point = Point(self.x, self.y)
        end_point = Point((self.x + move_x) % WIDTH, (self.y + move_y) % HEIGHT)
        self.x = (self.x + move_x) % WIDTH
        self.y = (self.y + move_y) % HEIGHT
        self.hunger -= 1
        if self.hunger < 3:
            self.is_hungry = True
        return start_point, end_point

    def reproduce(self, predators, entities):
        child_x = random.randint(self.x - 1, self.x + 1)
        child_y = random.randint(self.y - 1, self.y + 1)
        while (child_x, child_y) == (self.x, self.y):
            child_x = random.randint(self.x - 1, self.x + 1)
            child_y = random.randint(self.y - 1, self.y + 1)

            entities.append(Star(child_x, child_y, self.field_of_view, self.run_length))
            predators.append(Star(child_x, child_y, self.field_of_view, self.run_length))
    
    def is_alive(self):
        return self.hunger > 0


class Circle(Point):
    def __init__(self, x, y, filed_of_view, run_length):
        super().__init__(x, y)
        self.hunger = random.randint(10, 20)
        self.is_hungry = False#random.choice([True, False])
        self.field_of_view = filed_of_view
        self.run_length = run_length

    def detect_plant(self, plant):
        detected = []
        for entity in plant:
            dx = self.x - entity.x
            dy = self.y - entity.y
            if abs(dx) < self.field_of_view and abs(dy) < self.field_of_view:
                detected.append(entity)
        return detected

    def detect_partner(self, predators):
        detected = []
        for entity in predators:
            dx = self.x - entity.x
            dy = self.y - entity.y
            if abs(dx) < self.field_of_view and abs(dy) < self.field_of_view and (self.x != entity.x and self.y != entity.y):
                detected.append(entity)
        return detected

    def move(self, plant, prey, predators, entities):
        move_x = 0
        move_y = 0
        min_x = 10 ** 6
        min_y = 10 ** 6
        if self.is_hungry:
            detected_plant = self.detect_plant(plant)
            pot_plant = None
            for entity in detected_plant:
                dx = self.x - entity.x
                dy = self.y - entity.y
                if abs(dx) < abs(min_x) and abs(dx) < abs(min_y):
                    min_x = dx
                    min_y = dy
                    pot_plant = entity
    
            if pot_plant is None:
                move_x, move_y = random.choice([(0, self.run_length), (self.run_length, 0), (0, -self.run_length), (-self.run_length, 0)])
                
            elif pot_plant and pot_plant.is_alive():
                move_x = - np.sign(min_x) * self.run_length
                move_y = - np.sign(min_y) * self.run_length

                if abs(dx) <= 1 and abs(dy) <= 1:
                    pot_plant.max_age = 0
                    self.is_hungry = False
                    self.hunger = 5
                    self.x = pot_plant.x
                    self.y = pot_plant.y
        else:
            detected_partners = self.detect_partner(prey)
            pot_partner = None
            for entity in detected_partners:
                dx = self.x - entity.x
                dy = self.y - entity.y
                if abs(dx) < abs(min_x) and abs(dx) < abs(min_y):
                    min_x = dx
                    min_y = dy
                    pot_partner = entity
            if pot_partner is None:
                move_x, move_y = random.choice([(0, self.run_length), (self.run_length, 0), (0, -self.run_length), (-self.run_length, 0)])

            elif pot_partner and pot_partner.is_alive():
                move_x = - np.sign(min_x) * self.run_length
                move_y = - np.sign(min_y) * self.run_length

                if abs(dx) <= 1 and abs(dy) <= 1:
                    self.reproduce(prey, entities)
        start_point = Point(self.x, self.y)
        end_point = Point((self.x + move_x) % WIDTH, (self.y + move_y) % HEIGHT)
        self.x = (self.x + move_x) % WIDTH
        self.y = (self.y + move_y) % HEIGHT
        self.hunger -= 1
        if self.hunger < 3:
            self.is_hungry = True
        return start_point, end_point

    def reproduce(self, prey, entities):
        child_x = random.randint(self.x - 1, self.x + 1)
        child_y = random.randint(self.y - 1, self.y + 1)
        while (child_x, child_y) == (self.x, self.y):
            child_x = random.randint(self.x - 1, self.x + 1)
            child_y = random.randint(self.y - 1, self.y + 1)

            entities.append(Circle(child_x, child_y, self.field_of_view, self.run_length))
            prey.append(Circle(child_x, child_y, self.field_of_view, self.run_length))
    
    def is_alive(self):
        return self.hunger > 0

In [292]:
WIDTH = 100
HEIGHT = 100
number_of_plants = 10
number_of_predators = 10
number_of_prey = 20

In [293]:
plants = []
predators = []
preys = []
occ_points = []

for _ in range(number_of_plants):
    x = random.randint(0, WIDTH - 1)
    y = random.randint(0, HEIGHT - 1)
    plants.append(Triangle(x, y, 5, 5, 100))
    occ_points.append((x, y))

for _ in range(number_of_predators):
    x = random.randint(0, WIDTH - 1)
    y = random.randint(0, HEIGHT - 1)
    while (x, y) in occ_points:
        x = random.randint(0, WIDTH - 1)
        y = random.randint(0, HEIGHT - 1)
    predators.append(Star(x, y, 3, 5))
    occ_points.append((x, y))

for _ in range(number_of_prey):
    x = random.randint(0, WIDTH - 1)
    y = random.randint(0, HEIGHT - 1)
    while (x, y) in occ_points:
        x = random.randint(0, WIDTH - 1)
        y = random.randint(0, HEIGHT - 1)
    preys.append(Circle(x, y, 7, 3))
    occ_points.append((x, y))

In [294]:
print(f"Amount: {len(plants)}")
for entity in plants:
    print("Plant --------")
    print(entity.x, entity.y, entity.reproduce_timer, entity.max_age, entity.is_alive())

print(f"Amount: {len(predators)}")
for entity in predators:
    print("Predator --------")
    print(entity.x, entity.y, entity.hunger, entity.is_hungry, entity.is_alive())

print(f"Amount: {len(prey)}")
for entity in preys:
    print("Prey --------")
    print(entity.x, entity.y, entity.hunger, entity.is_hungry, entity.is_alive())

Amount: 10
Plant --------
18 87 5 100 True
Plant --------
64 49 5 100 True
Plant --------
2 29 5 100 True
Plant --------
30 44 5 100 True
Plant --------
86 61 5 100 True
Plant --------
38 13 5 100 True
Plant --------
28 31 5 100 True
Plant --------
64 31 5 100 True
Plant --------
77 53 5 100 True
Plant --------
49 16 5 100 True
Amount: 10
Predator --------
89 21 10 False True
Predator --------
38 64 20 False True
Predator --------
77 55 23 False True
Predator --------
31 25 20 False True
Predator --------
77 86 19 False True
Predator --------
9 11 28 False True
Predator --------
9 40 21 False True
Predator --------
78 26 11 False True
Predator --------
20 64 28 False True
Predator --------
59 25 29 False True
Amount: 1
Prey --------
64 64 19 False True
Prey --------
48 13 13 False True
Prey --------
33 80 10 False True
Prey --------
50 86 14 False True
Prey --------
95 36 15 False True
Prey --------
9 0 17 False True
Prey --------
82 36 16 False True
Prey --------
86 96 17 False True
P

In [295]:
fig, ax = plt.subplots()
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
plants_scatter = ax.scatter([], [], c='green', label='Plants')
preys_scatter = ax.scatter([], [], c='blue', label='Preys')
predators_scatter = ax.scatter([], [], c='red', label='Predators')

<IPython.core.display.Javascript object>

In [296]:
def update(frame):
    # Симуляция изменений в игре
    for plant in plants:
        plant.reproduce(plants, plants + preys + predators)
    for prey in preys:
        prey.move(plants, preys, predators, preys + predators)
    for predator in predators:
        predator.move(plants, preys, predators, preys + predators)

    # Проверка структуры данных
    plants_alive = [(plant.x, plant.y) for plant in plants if plant.is_alive()]
    preys_alive = [(prey.x, prey.y) for prey in preys if prey.is_alive()]
    predators_alive = [(predator.x, predator.y) for predator in predators if predator.is_alive()]

    # Обновляем данные для анимации только если есть живые существа
    if plants_alive:  # Если есть живые растения
        plants_scatter.set_offsets(np.array(plants_alive))
    else:
        plants_scatter.set_offsets(np.empty((0, 2)))  # Пустой двумерный массив

    if preys_alive:  # Если есть живые травоядные
        preys_scatter.set_offsets(np.array(preys_alive))
    else:
        preys_scatter.set_offsets(np.empty((0, 2)))  # Пустой двумерный массив

    if predators_alive:  # Если есть живые хищники
        predators_scatter.set_offsets(np.array(predators_alive))
    else:
        predators_scatter.set_offsets(np.empty((0, 2)))  # Пустой двумерный массив

    # Удаляем старый текст и добавляем новый с номером текущего фрейма
    for text in ax.texts:
        text.remove()

    # Добавляем новый текст с номером фрейма
    ax.text(0.5, 0.95, f"Frame: {frame}", ha="center", va="top", transform=ax.transAxes, fontsize=12, color='black')

    return plants_scatter, preys_scatter, predators_scatter


In [297]:
ani = FuncAnimation(fig, update, frames=70, interval=200, blit=True)

HTML(ani.to_jshtml())

In [ ]:
for epoch in range(10):
    for entity in (plants + predators + prey):
        if not isinstance(entity, Triangle):
            entity.move(plants, prey, predators, plants + predators + prey)
        if isinstance(entity, Triangle):
            entity.reproduce(plants, (plants + predators + prey))
            entity.max_age -= 1
        if entity.is_alive() == False:
            if isinstance(entity, Triangle):
                plants.remove(entity)
            elif isinstance(entity, Star):
                predators.remove(entity)
            elif isinstance(entity, Circle):
                prey.remove(entity)

    print(f"Epoch: ---------{epoch}---------")
    print(f"Amount: {len(plants)}")
    for entity in plants:
        print("Plant --------")
        print(entity.x, entity.y, entity.reproduce_timer, entity.max_age, entity.is_alive())
    print(f"Amount: {len(predators)}")
    for entity in predators:
        print("Predator --------")
        print(entity.x, entity.y, entity.hunger, entity.is_hungry)

    print(f"Amount: {len(prey)}")
    for entity in prey:
        print("Prey --------")
        print(entity.x, entity.y, entity.hunger, entity.is_hungry)